**Brazil's House of Deputies Reimbursements**

This notebook, makes an exploratory analysis of data, in the individual and collective expenses of the Brazilian deputies.


![](http://www.vermelho.org.br/admin/arquivos/biblioteca/cd99755.jpg)

In [108]:
import pandas as pd

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

init_notebook_mode(connected=True)

**Load Data**

In [109]:
path2data = '../input/'

In [110]:
df_politicians = pd.read_csv(path2data + 'deputies_dataset.csv', low_memory=False)
df_dirty_politicians = pd.read_csv(path2data + 'dirty_deputies_v2.csv', low_memory=False)

In [111]:
print('Dataset shape: {}'.format(df_politicians.shape))

Dataset shape: (3014902, 10)


In [112]:
df_politicians['receipt_date'] = pd.to_datetime(df_politicians['receipt_date'])
df_politicians['month'] = df_politicians['receipt_date'].dt.month
df_politicians['year'] = df_politicians['receipt_date'].dt.year
df_politicians['day'] = df_politicians['receipt_date'].dt.day

In [113]:
df_politicians.head()

,bugged_date,receipt_date,deputy_id,political_party,state_code,deputy_name,receipt_social_security_number,receipt_description,establishment_name,receipt_value,month,year,day
0,0,2013-03-27,1772,PSB,SP,Abelardo Camarinha,3.530749e+12,Fuels and lubricants.,AUTO POSTO 314 NORTE LTDA,70,3,2013,27
1,0,2013-07-24,1772,PSB,SP,Abelardo Camarinha,8.202116e+12,Fuels and lubricants.,AUTO POSTO AEROPORTO LTDA,104,7,2013,24
2,0,2013-02-17,1772,PSB,SP,Abelardo Camarinha,8.202116e+12,Fuels and lubricants.,AUTO POSTO AEROPORTO LTDA,100,2,2013,17
3,0,2013-03-15,1772,PSB,SP,Abelardo Camarinha,8.202116e+12,Fuels and lubricants.,AUTO POSTO AEROPORTO LTDA,100,3,2013,15
4,0,2013-01-27,1772,PSB,SP,Abelardo Camarinha,8.202116e+12,Fuels and lubricants.,AUTO POSTO AEROPORTO LTDA,77,1,2013,27


In [114]:
# get data from 2013 to 2017
df_politicians = df_politicians[ (df_politicians['year']>=2013) &  (df_politicians['year']<=2017)]

In [115]:
df_dirty_politicians['receipt_date'] = pd.to_datetime(df_dirty_politicians['refund_date'])
df_dirty_politicians['month'] = df_dirty_politicians['receipt_date'].dt.month
df_dirty_politicians['year'] = df_dirty_politicians['receipt_date'].dt.year
df_dirty_politicians['day'] = df_dirty_politicians['receipt_date'].dt.day

In [120]:
df_dirty_politicians.head()

,deputy_name,deputy_state,political_party,refund_description,company_name,company_id,refund_date,refund_value,party_pg,party_en,party_tse,party_regdate,party_nmembers,party_ideology1,party_ideology2,party_ideology3,party_ideology4,party_position,receipt_date,month,year,day
0,PROFESSORA DORINHA SEABRA REZENDE,TO,DEM,FLIGHT TICKET,Cia Aérea - TAM,2.012862e+12,2016-07-29T00:00:00,937.63,Democratas,Democrats,25,11-09-86,1095712,Economic liberalism,Liberal conservatism,Christian democracy,NaN,Rightwing,2016-07-29,7.0,2016.0,29.0
1,JOSÉ CARLOS ALELUIA,BA,DEM,FUEL COSTS,CASCOL COMBUSTIVEIS PARA VEICULOS LTDA,3.065970e+11,2016-06-21T00:00:00,150.00,Democratas,Democrats,25,11-09-86,1095712,Economic liberalism,Liberal conservatism,Christian democracy,NaN,Rightwing,2016-06-21,6.0,2016.0,21.0
2,JOSÉ CARLOS ALELUIA,BA,DEM,FUEL COSTS,CASCOL COMBUSTIVEIS PARA VEICULOS LTDA,3.065970e+11,2016-06-01T00:00:00,100.00,Democratas,Democrats,25,11-09-86,1095712,Economic liberalism,Liberal conservatism,Christian democracy,NaN,Rightwing,2016-06-01,6.0,2016.0,1.0
3,JOSÉ CARLOS ALELUIA,BA,DEM,FUEL COSTS,MEGA POSTO BERIMBAU LTDA - ME,1.855960e+12,2016-04-19T00:00:00,168.07,Democratas,Democrats,25,11-09-86,1095712,Economic liberalism,Liberal conservatism,Christian democracy,NaN,Rightwing,2016-04-19,4.0,2016.0,19.0
4,JOSÉ CARLOS ALELUIA,BA,DEM,FUEL COSTS,MEGA POSTO BERIMBAU LTDA - ME,1.855960e+12,2016-08-13T00:00:00,239.57,Democratas,Democrats,25,11-09-86,1095712,Economic liberalism,Liberal conservatism,Christian democracy,NaN,Rightwing,2016-08-13,8.0,2016.0,13.0


**Exploratory Data Analysis**

**Receipt values by deputies of each state and Political**

As can be seen from the chart below, most of the expenses are from deputies from the state of Sao Paulo. With regard to the political party, the one that had the highest expenses in the analyzed period was the PT (146M).

In [85]:
trace1 = go.Bar(
            x=df_politicians.groupby(['state_code'])['receipt_value'].sum().sort_values(ascending=True).values,
            y=df_politicians.groupby(['state_code'])['receipt_value'].sum().sort_values(ascending=True).index,
            orientation = 'h'
)

trace2 = go.Bar(
            x=df_politicians.groupby(['political_party'])['receipt_value'].sum().sort_values(ascending=True).values,
            y=df_politicians.groupby(['political_party'])['receipt_value'].sum().sort_values(ascending=True).index,
            orientation = 'h'
)

fig = tools.make_subplots(rows=1, cols=2)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout'].update(height=600, width=1000, title='Receipt Value counter')
iplot(fig, filename='simple-subplot-with-annotations')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [86]:
df_politicians.groupby(['deputy_name', 'political_party'])['receipt_value'].sum().sort_values(ascending=True)[-10:]

deputy_name           political_party
Vander Loubet         PT                 2541904
Wellington Roberto    PR                 2610286
Silas Câmara          PRB                2614327
Waldir Maranhão       AVANTE             2628099
Alice Portugal        PCDOB              2669018
Zé Geraldo            PT                 2674437
Bonifácio De Andrada  PSDB               2681787
Paes Landim           PTB                2857957
Edio Lopes            PR                 3001949
Felipe Bornier        PROS               3090180
Name: receipt_value, dtype: int64

# Individual analysis

In [125]:
deputy_name = 'Felipe Bornier' # change here to analyze other deputy

In [126]:
deputy_data = df_politicians[df_politicians['deputy_name'] == deputy_name]

In [127]:
deputy_data.head()

,bugged_date,receipt_date,deputy_id,political_party,state_code,deputy_name,receipt_social_security_number,receipt_description,establishment_name,receipt_value,month,year,day
113194,0,2013-01-02,1831,PROS,RJ,Felipe Bornier,5.239115e+12,Dissemination of the Parliamentary Activity.,MARWIC ARTES GRAFICAS LTDA,12000,1,2013,2
113195,0,2013-02-01,1831,PROS,RJ,Felipe Bornier,5.239115e+12,Dissemination of the Parliamentary Activity.,MARWIC ARTES GRAFICAS LTDA,12000,2,2013,1
113196,0,2013-04-01,1831,PROS,RJ,Felipe Bornier,5.239115e+12,Dissemination of the Parliamentary Activity.,MARWIC ARTES GRAFICAS LTDA,14000,4,2013,1
113197,0,2013-05-02,1831,PROS,RJ,Felipe Bornier,5.239115e+12,Dissemination of the Parliamentary Activity.,MARWIC ARTES GRAFICAS LTDA,16000,5,2013,2
113198,0,2013-07-01,1831,PROS,RJ,Felipe Bornier,5.239115e+12,Dissemination of the Parliamentary Activity.,MARWIC ARTES GRAFICAS LTDA,18000,7,2013,1


**Trajectory of receipt value in each month**

As can be seen, December is the month in which the number of expenses is higher

In [90]:
receipt_value_month_year = deputy_data.groupby(['year','month'])['receipt_value'].sum().to_frame().unstack(level=-1)
receipt_value_month_year.columns = receipt_value_month_year.columns.droplevel(0)

In [91]:
receipt_value_month_year.head()

month,1,2,3,4,5,6,7,8,9,10,11,12
year,,,,,,,,,,,,
2013,30230.0,32823.0,40402.0,30635.0,35631.0,41305.0,48117.0,55649.0,42898.0,47264.0,52488.0,73787.0
2014,38142.0,38481.0,54821.0,79463.0,60735.0,35606.0,34994.0,15636.0,9652.0,52836.0,62351.0,119202.0
2015,44488.0,43932.0,62885.0,88620.0,70671.0,57644.0,55308.0,74634.0,71459.0,52178.0,53890.0,87995.0
2016,31335.0,45732.0,91149.0,62582.0,67990.0,49639.0,50470.0,89071.0,62714.0,60100.0,62016.0,94577.0
2017,42973.0,50189.0,44920.0,50884.0,47803.0,51438.0,26467.0,68685.0,22825.0,19769.0,NaN,NaN


In [92]:
data = []
years = receipt_value_month_year.index

for idx in range(0, len(years)):

    trace = go.Bar(
        x=receipt_value_month_year.iloc[0].index,
        y=receipt_value_month_year.iloc[idx],
        name=str(years[idx])
    )
    data.append(trace)
    
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
fig['layout'].update(height=600, width=1000, title='Receipt Value in each month')
iplot(fig, filename='grouped-bar')

In [93]:
deputy_data['receipt_description'].unique()

array(['Dissemination of the Parliamentary Activity.', 'Telephony',
       'Postal Services', 'Airline tickets',
       'Consultancies, Researches and Technical Works.',
       'Maintenance of Office'], dtype=object)

**Distribution of paid services**



In [94]:
labels = deputy_data.groupby(['receipt_description'])['receipt_value'].sum().index
values = deputy_data.groupby(['receipt_description'])['receipt_value'].sum().values

trace = go.Pie(labels=labels, values=values)
iplot([trace], filename='basic_pie_chart')

# Airline Tickets

In [95]:
deputy_data[deputy_data['receipt_description'] == 'Airline tickets'].groupby(['establishment_name'])['receipt_value'].sum().sort_values(ascending=False)

establishment_name
Cia Aérea - GOL          1436734
Cia Aérea - TAM           305146
Cia Aérea - AVIANCA       124308
Cia Aérea - AZUL            9076
American Airlines           4544
Cia Aérea - PASSAREDO        691
Name: receipt_value, dtype: int64

In [96]:
deputy_airline_df = deputy_data[deputy_data['receipt_description'] == 'Airline tickets']

In [97]:
data = [go.Scatter(
          x=deputy_airline_df['receipt_date'],
          y=deputy_airline_df['receipt_value'],
          mode = 'markers'
    )]

iplot(data)

# Number of trips by day

As can be seen below, there are deputies who have made many trips in just one day.

In [98]:
nb_trips = deputy_airline_df.groupby(['receipt_date'])['receipt_date'].count().sort_values(ascending=False)

In [99]:
nb_trips.head(10)

receipt_date
2013-05-09    21
2015-08-21    21
2016-03-18    20
2015-08-07    19
2015-04-16    18
2013-08-09    17
2016-12-16    16
2016-04-01    16
2014-05-21    15
2013-10-29    15
Name: receipt_date, dtype: int64

In [100]:
deputy_airline_df[deputy_airline_df['receipt_date'] == nb_trips.index[0]]

,bugged_date,receipt_date,deputy_id,political_party,state_code,deputy_name,receipt_social_security_number,receipt_description,establishment_name,receipt_value,month,year,day
113437,0,2013-05-09,1831,PROS,RJ,Felipe Bornier,7.575651e+12,Airline tickets,Cia Aérea - GOL,359,5,2013,9
113448,0,2013-05-09,1831,PROS,RJ,Felipe Bornier,7.575651e+12,Airline tickets,Cia Aérea - GOL,310,5,2013,9
113506,0,2013-05-09,1831,PROS,RJ,Felipe Bornier,7.575651e+12,Airline tickets,Cia Aérea - GOL,503,5,2013,9
113539,0,2013-05-09,1831,PROS,RJ,Felipe Bornier,7.575651e+12,Airline tickets,Cia Aérea - GOL,464,5,2013,9
113549,0,2013-05-09,1831,PROS,RJ,Felipe Bornier,7.575651e+12,Airline tickets,Cia Aérea - GOL,464,5,2013,9
113569,0,2013-05-09,1831,PROS,RJ,Felipe Bornier,7.575651e+12,Airline tickets,Cia Aérea - GOL,446,5,2013,9
113649,0,2013-05-09,1831,PROS,RJ,Felipe Bornier,7.575651e+12,Airline tickets,Cia Aérea - GOL,243,5,2013,9
113710,0,2013-05-09,1831,PROS,RJ,Felipe Bornier,7.575651e+12,Airline tickets,Cia Aérea - GOL,446,5,2013,9
113761,0,2013-05-09,1831,PROS,RJ,Felipe Bornier,7.575651e+12,Airline tickets,Cia Aérea - GOL,453,5,2013,9
113762,0,2013-05-09,1831,PROS,RJ,Felipe Bornier,7.575651e+12,Airline tickets,Cia Aérea - GOL,136,5,2013,9


# Postal Services

In [101]:
aux = deputy_data[deputy_data['receipt_description'] == 'Postal Services'].groupby(['establishment_name'])['receipt_value'].sum().to_frame()
aux.sort_values('receipt_value', ascending=False, inplace=True)

In [102]:
aux

,receipt_value
establishment_name,
CORREIOS - CARTA COMERCIAL,126629
CORREIOS - SEDEX CONVENCIONAL,14632
CORREIOS - SEDEX 10,5369
CORREIOS - ENCOMENDA PAC,2678
CORREIOS - SEDEX 12,2127
CORREIOS - SEDEX 10 COM AR,569
CORREIOS - SEDEX CONVENCIONAL COM AR,557
CORREIOS - CAIXA DE ENCOMENDAS BÁSICA/CONVENCIONAL,4
CORREIOS - CARTA REGISTRADA,4


In [103]:
types_of_postal_services = deputy_data[deputy_data['receipt_description'] == 'Postal Services']['establishment_name'].unique()

In [104]:
print('Types of postal services: {}'.format(len(types_of_postal_services)))

Types of postal services: 9


In [105]:
data = []
for ps in types_of_postal_services:

    aux = deputy_data[(deputy_data['receipt_description'] == 'Postal Services') & (deputy_data['establishment_name'] == ps)]
    trace = go.Box(
        x=aux['receipt_value'].values,
        name = ps
    )
    
    data.append(trace)

layout = go.Layout(
    width=800,
    yaxis=dict(
        zeroline=False
    ),
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

# Dissemination of the Parliamentary Activity.

In [106]:
dpa = deputy_data[deputy_data['receipt_description'] == 'Dissemination of the Parliamentary Activity.'].groupby(['establishment_name', 'year'])['receipt_value'].sum().to_frame().unstack(level=-1)
dpa.columns = dpa.columns.droplevel(0)

In [107]:
dpa

year,2013,2014,2015,2016,2017
establishment_name,,,,,
MARWIC ARTES GRAFICAS LTDA,194000.0,18000.0,32000.0,192000.0,112000.0
cotton Brasil Publicidade,NaN,NaN,NaN,NaN,52000.0
marwic artes graficas ltda,NaN,204000.0,162000.0,NaN,NaN
